In [ ]:
import os
import numpy as np

In [2]:
jax_data_dir = "/global/scratch/users/aniketh/promoter_modelling/jax_data/"
scripts_dir = "/global/home/users/aniketh/promoter/"

oracle_test_data_path = os.path.join(jax_data_dir, "finetune_data.pkl")
assert os.path.exists(oracle_test_data_path)

predictions_save_dir = os.path.join(jax_data_dir, "oracle_predictions_v7_TITAN")
if not os.path.exists(predictions_save_dir):
    os.mkdir(predictions_save_dir)
assert os.path.exists(predictions_save_dir)

bash_scripts_dir = "/global/home/users/aniketh/promoter/promoter/get_oracle_predictions_bash_scripts_v7_TITAN"
if not os.path.exists(bash_scripts_dir):
    os.mkdir(bash_scripts_dir)
    
total_count = len(os.listdir(os.path.join(jax_data_dir, "trained_predictors_new_loss")))
    
num_splits = 1
runs_per_job = int(np.ceil(total_count / num_splits))
job_cnt = 0
cur_cnt = 0

slurm_prefix = open("slurm_prefix.txt", "r").readlines()

g = open(os.path.join(bash_scripts_dir, f"get_all_predictions_j{job_cnt}.sh"), "w+")
for line in slurm_prefix:
    g.write(line)
    
rem_count = 0

for model in sorted(os.listdir(os.path.join(jax_data_dir, "trained_predictors_new_loss"))):
    pretrained_predictor_path = os.path.join(jax_data_dir, "trained_predictors_new_loss", model)
            
    if os.path.exists(os.path.join(predictions_save_dir, model, "oracle_test_ground_truth.npy")):
        continue
                
    rem_count += 1
        
    if cur_cnt == runs_per_job:
        job_cnt += 1
        cur_cnt = 0
        g.close()
        g = open(os.path.join(bash_scripts_dir, f"get_all_predictions_j{job_cnt}.sh"), "w+")
        for line in slurm_prefix:
            g.write(line)
        print("Created new job split")

    cmd = f"sbatch --requeue --export=scripts_dir={scripts_dir},pretrained_predictor_path={pretrained_predictor_path},oracle_test_data_path={oracle_test_data_path},predictions_save_dir={predictions_save_dir} get_finetune_data_predictions.sh"
    os.system(cmd)
    print(cmd)
    print()

    f = open(os.path.join(bash_scripts_dir, model + ".sh"), "w+")
    f.write("#!/bin/bash\n")
    all_exports = f"scripts_dir={scripts_dir},pretrained_predictor_path={pretrained_predictor_path},oracle_test_data_path={oracle_test_data_path},predictions_save_dir={predictions_save_dir}"
    for ex in all_exports.split(","):
        var, val = ex.split("=")
        print(f"export {var}={val}")
        f.write(f"export {var}={val}\n")
    print("bash {}".format(os.path.join(scripts_dir, "promoter/get_finetune_data_predictions.sh")))
    f.write("bash {}\n".format(os.path.join(scripts_dir, "promoter/get_finetune_data_predictions.sh")))
    f.close()

    g.write("bash {}".format(os.path.join(bash_scripts_dir, model + ".sh\n")))        
    print()

    cur_cnt += 1
    
g.close()

In [3]:
total_count

8

In [4]:
rem_count

0